In [104]:
import torch as T
from torch.jit import Error
import torch.nn as nn
import torch.optim as optim
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F

class TNet(nn.Module):
    def __init__(self, n_feats, n_samples):
        super(TNet, self).__init__()
        self.n_feats = n_feats
        self.conv1 = nn.Conv1d(n_feats, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, n_feats*n_feats)
        self.bn1 = nn.LayerNorm(T.Size([64, n_samples]))
        self.bn2 = nn.LayerNorm(T.Size([128, n_samples]))
        self.bn3 = nn.LayerNorm(T.Size([1024,n_samples]))
        self.bn4 = nn.LayerNorm(512)
        self.bn5 = nn.LayerNorm(256)
        
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, x):
        batch_size = x.size()[0]
        bias = Variable(torch.from_numpy(np.eye(self.n_feats).flatten().astype(np.float32))).view(1,self.n_feats*self.n_feats).repeat(batch_size,1).to(self.device)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))

        x = nn.MaxPool1d(x.size(-1))(x)
        x = nn.Flatten(1)(x)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        x = x + bias
        x = x.view(-1, self.n_feats, self.n_feats)
        return x


class PointNet(nn.Module):
    def __init__(self, mode='classifier', n_samples=1024, n_classes=10):
        super(PointNet, self).__init__()
        self.in_trans = TNet(3, n_samples)
        self.feat_trans = TNet(64, n_samples)
        self.mode = mode
        self.n_samples = n_samples
        self.n_classes = n_classes

        if(self.mode!='classifier' and self.mode!='segmentation'):
            raise Exception("wrong mode selected! choose 'classifier' or 'segmentation'")

        self.mlp1 = nn.Sequential(nn.Conv1d(3, 64, 1),
                                nn.LayerNorm(T.Size([64, n_samples])),
                                nn.ReLU(),
                                nn.Conv1d(64, 64, 1),
                                nn.LayerNorm(T.Size([64, n_samples])),
                                nn.ReLU())

        self.mlp2 = nn.Sequential(nn.Conv1d(64, 64, 1),
                                nn.LayerNorm(T.Size([64, n_samples])),
                                nn.ReLU(),
                                nn.Conv1d(64, 128, 1),
                                nn.LayerNorm(T.Size([128, n_samples])),
                                nn.ReLU(),
                                nn.Conv1d(128, 1024, 1),
                                nn.LayerNorm(T.Size([1024, n_samples])),
                                nn.ReLU())

        self.mlp3 = nn.Sequential(nn.Linear(1024, 512),
                                    nn.LayerNorm(512),
                                    nn.ReLU(),
                                    nn.Linear(512, 256),
                                    nn.LayerNorm(256),
                                    nn.ReLU(),
                                    nn.Dropout(0.3),
                                    nn.Linear(256, n_classes),
                                    nn.Softmax(1))

        self.mlp4 = nn.Sequential(nn.Conv1d(1088, 512, 1),
                                nn.LayerNorm(T.Size([512, n_samples])),
                                nn.ReLU(),
                                nn.Conv1d(512, 256, 1),
                                nn.LayerNorm(T.Size([256, n_samples])),
                                nn.ReLU(),
                                nn.Conv1d(256, 128, 1),
                                nn.LayerNorm(T.Size([128, n_samples])),
                                nn.ReLU())

        self.mlp5 = nn.Sequential(nn.Conv1d(128, n_classes, 1),
                                nn.LayerNorm(T.Size([n_classes, n_samples])),
                                nn.ReLU(),
                                nn.Softmax(1))

        # self.optimizer = optim.Adam(self.parameters(), lr=0.001, betas=(0.9, 0.999))

        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, x):
        x = x.to(self.device)

        input_transform = self.in_trans.forward(T.transpose(x,1,2))
        x = T.bmm(x, input_transform) # nx3

        x = T.transpose(self.mlp1(T.transpose(x,1,2)), 1, 2) # nx64
        feat_transform = self.feat_trans(T.transpose(x,1,2))
        x_to_concat = T.bmm(x, feat_transform) # nx64

        x = T.transpose(self.mlp2(T.transpose(x_to_concat,1,2)), 1, 2) # nx1024
        global_feat = T.max(x, 1, keepdim=True)[0].view(-1, 1024)

        if self.mode == 'classifier':
            x = self.mlp3(global_feat)
            return x, feat_transform

        elif self.mode == 'segmentation':
            x = global_feat.view(-1, 1, 1024).repeat(1, self.n_samples, 1)
            x = T.cat([x_to_concat, x], 2)
            point_feats = T.transpose(self.mlp4(T.transpose(x,1,2)), 1, 2)
            x = T.transpose(self.mlp5(T.transpose(point_feats,1,2)), 1, 2)
            return x

In [106]:
net = PointNet(mode='segmentation', n_samples=10, n_classes=10).to(T.device('cuda:0'))
x = T.rand(32,10,3).to(T.device('cuda:0'))
x = net(x)
x[0]

tensor([[0.0593, 0.0593, 0.0593, 0.2791, 0.0593, 0.0820, 0.0593, 0.0593, 0.0941,
         0.1888],
        [0.0522, 0.0522, 0.0522, 0.2956, 0.0674, 0.0835, 0.0522, 0.0522, 0.1251,
         0.1674],
        [0.0515, 0.0515, 0.0515, 0.2600, 0.0747, 0.0947, 0.0515, 0.0567, 0.1653,
         0.1424],
        [0.0579, 0.0579, 0.0579, 0.2371, 0.0662, 0.0958, 0.0579, 0.0579, 0.1278,
         0.1837],
        [0.0607, 0.0607, 0.0607, 0.2247, 0.0658, 0.0917, 0.0607, 0.0607, 0.0996,
         0.2145],
        [0.0438, 0.0438, 0.0438, 0.3010, 0.0785, 0.1091, 0.0438, 0.0438, 0.1870,
         0.1054],
        [0.0564, 0.0564, 0.0564, 0.2599, 0.0672, 0.0861, 0.0564, 0.0748, 0.1259,
         0.1605],
        [0.0504, 0.0504, 0.0504, 0.2669, 0.0718, 0.0958, 0.0504, 0.0504, 0.1672,
         0.1463],
        [0.0606, 0.0606, 0.0606, 0.2353, 0.0687, 0.0782, 0.0606, 0.0606, 0.0870,
         0.2279],
        [0.0594, 0.0594, 0.0594, 0.2435, 0.0656, 0.0773, 0.0594, 0.0594, 0.0860,
         0.2305]], device='c

In [52]:
a = T.rand(1,10, 1024)
a = torch.max(a, 1, keepdim=True)[0]
a.shape

torch.Size([1, 1, 1024])

In [98]:
x = T.rand(32, 10, 15)
x = T.transpose(nn.Softmax(1)(x), 1, 2)
x[0]

tensor([[0.0686, 0.0737, 0.0573, 0.0617, 0.0930, 0.0412, 0.0974, 0.0720, 0.0720,
         0.0791],
        [0.0915, 0.0724, 0.0474, 0.0399, 0.0515, 0.0858, 0.0629, 0.0499, 0.0626,
         0.0934],
        [0.0848, 0.0594, 0.0431, 0.1072, 0.0885, 0.0718, 0.0462, 0.0586, 0.0489,
         0.0780],
        [0.0682, 0.0459, 0.0596, 0.0494, 0.1031, 0.0664, 0.0581, 0.0925, 0.0912,
         0.0740],
        [0.0630, 0.0606, 0.0806, 0.0496, 0.0394, 0.0604, 0.0487, 0.0692, 0.0647,
         0.0910],
        [0.0342, 0.0470, 0.0575, 0.0838, 0.0730, 0.0868, 0.0561, 0.0746, 0.0478,
         0.0664],
        [0.0460, 0.0758, 0.1036, 0.0470, 0.0405, 0.0753, 0.0965, 0.0894, 0.0470,
         0.0781],
        [0.0861, 0.0954, 0.0505, 0.0867, 0.0433, 0.0620, 0.0705, 0.0596, 0.0936,
         0.0499],
        [0.0419, 0.0619, 0.0882, 0.0875, 0.1023, 0.0461, 0.0682, 0.0408, 0.0812,
         0.0550],
        [0.0520, 0.0925, 0.0869, 0.0908, 0.0403, 0.0619, 0.0474, 0.0718, 0.0832,
         0.0404],
        [0

In [107]:
np.sum(np.array([0.0593, 0.0593, 0.0593, 0.2791, 0.0593, 0.0820, 0.0593, 0.0593, 0.0941,
         0.1888]))

0.9998